In [ ]:
import json

import numpy as np
import pandas as pd

from dimod import ExactCQMSolver, ConstrainedQuadraticModel, QuadraticModel, Integer
from dwave.system import LeapHybridCQMSampler

In [ ]:
with open('tokens.json') as json_file:
    token = json.load(json_file)['dwave_token']